In [1]:
import tweepy
import csv
import numpy as np
from textblob import TextBlob
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# Connect to twitter
consumer_key = 'I3I5BDW4ttDCr557RAdeyuazw'
secret_key = 'N42CdUoS58HWlunwZmlLh3GfgmJq0L3VDm9rOLI5pgfQARKb3P'
access_token = '1223331047691243520-nnAuZuC0LGrk0iBxqHCF2KmVhaR63U'
access_token_secret = 'vCimNaIpryf0LCnIeLcV5IiZXrxBEXaAInOwD811IWgWs'
auth = tweepy.OAuthHandler(consumer_key, secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [10]:
# Fuction declarations


def get_data(filename):
    """
    Read opening prices for each day.
    """
    
    with open(filename, 'r') as csv_file:
        csv_file_reader = csv.reader(csv_file)
        next(csv_file_reader)
        for row in csv_file_reader:
            dates.append(row[0].replace('-',''))
            prices.append(float(row[1]))
            
        # Divide prices into training and test dataset
        train_size = int(len(dates) * 0.75)
        test_size = len(dates) - train_size
        train_dates, train_prices = dates[:train_size], prices[:train_size]
        test_dates, test_prices = dates[train_size:], prices[train_size:]       

    return train_dates, train_prices, test_dates, test_prices


def create_price_matrix(prices, look_back):
    """
    Create price matrix where dataset X is the price at a given time (t) 
    and dataset Y is the price at the next time (t + look_back).
    """
    priceX, priceY = [],[]
    
    for i in range(len(prices)-look_back):
        priceX.append(prices[i])
        priceY.append(prices[i+look_back])
    
    return np.array(priceX), np.array(priceY)


def build_model(train_priceX, train_priceY, test_priceX, test_priceY):
    """
    Builds, trains and evaluates a neural network model using Keras. 
    """
    
    # create and fit Multilayer Perceptron model
    model = Sequential()
    model.add(Dense(8, input_dim=look_back, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(train_priceX, train_priceY, epochs=200, batch_size=2, verbose=2)

    # Estimate model performance - Square root to get the avarage error
    train_score = model.evaluate(train_priceX, train_priceY, verbose=0)
    print('Train Score: %.2f MSE (%.2f RMSE)' % (train_score, np.sqrt(train_score)))
    test_score = model.evaluate(test_priceX, test_priceY, verbose=0)
    print('Test Score: %.2f MSE (%.2f RMSE)' % (test_score, np.sqrt(test_score)))
    
    return model

In [12]:
# MAIN

# Search for your company name on Twitter
public_tweets = api.search('Microsoft')

# Calculates the regression if the sentiment analysis returns positive.
threshold = 0.2
significant_sentiments = []

for tweet in public_tweets:    
    analysis = TextBlob(tweet.text)
    if abs(analysis.sentiment[0]) >= threshold:
        significant_sentiments.append(analysis.sentiment[0])
        
value = float()
for sentiment in significant_sentiments:
    value += sentiment
    
trigger_value = value/len(significant_sentiments)

if trigger_value:
    
    dates = []
    prices = []
    train_dates, train_prices, test_dates, test_prices = get_data('MSFT2.csv')

    look_back = 1
    train_priceX, train_priceY = create_price_matrix(train_prices, look_back)
    test_priceX, test_priceY = create_price_matrix(test_prices, look_back)
    
    model = build_model(train_priceX, train_priceY, test_priceX, test_priceY)
    
    prediction = model.predict(train_priceX)
    print('\nPositive sentiment analyis - regression values:')
    print(prediction)
    
else:
    print('\nNegative sentiment analyis - no regression.')

Epoch 1/200
 - 0s - loss: 7471.0880
Epoch 2/200
 - 0s - loss: 3969.9698
Epoch 3/200
 - 0s - loss: 1710.2361
Epoch 4/200
 - 0s - loss: 555.3135
Epoch 5/200
 - 0s - loss: 129.6716
Epoch 6/200
 - 0s - loss: 23.0264
Epoch 7/200
 - 0s - loss: 4.9878
Epoch 8/200
 - 0s - loss: 2.7637
Epoch 9/200
 - 0s - loss: 2.5422
Epoch 10/200
 - 0s - loss: 2.5269
Epoch 11/200
 - 0s - loss: 2.5268
Epoch 12/200
 - 0s - loss: 2.5484
Epoch 13/200
 - 0s - loss: 2.5449
Epoch 14/200
 - 0s - loss: 2.5340
Epoch 15/200
 - 0s - loss: 2.5269
Epoch 16/200
 - 0s - loss: 2.5457
Epoch 17/200
 - 0s - loss: 2.5361
Epoch 18/200
 - 0s - loss: 2.5261
Epoch 19/200
 - 0s - loss: 2.5443
Epoch 20/200
 - 0s - loss: 2.5670
Epoch 21/200
 - 0s - loss: 2.5348
Epoch 22/200
 - 0s - loss: 2.5447
Epoch 23/200
 - 0s - loss: 2.5613
Epoch 24/200
 - 0s - loss: 2.5267
Epoch 25/200
 - 0s - loss: 2.5308
Epoch 26/200
 - 0s - loss: 2.5576
Epoch 27/200
 - 0s - loss: 2.6341
Epoch 28/200
 - 0s - loss: 2.5436
Epoch 29/200
 - 0s - loss: 2.5453
Epoch 30/